In [1]:
using Plots
gr()

Plots.GRBackend()

In [2]:
function wave_fixed(u0, v0, c, L, tf; N = 10, MinM = 1)
    h = L/(N-1)
    M = max(ceil(Int, c*tf/h) + 1, MinM)
    δ = tf/M
    κ = (c*δ/h)^2
    
    x = linspace(0, L, N)
    t = linspace(0, tf, M+1)
    V = zeros(N, M+1)
    
    # Initial position
    V[1,1] = 0
    V[N,1] = 0
    for i = 2:N-1
        V[i,1] = u0(x[i])
    end
    # Initial velocity
    V[1,2] = 0
    for i = 2:N-1
        V[i,2] = δ*v0(x[i]) + (κ*V[i+1,1] + 2(1-κ)*V[i,1] + κ*V[i-1,1])/2
    end
    V[N,2] = 0

    for n = 2:M
        V[1,n+1] = 0
        for i = 2:N-1
            V[i,n+1] = -V[i,n-1] + κ*V[i+1,n] + 2(1-κ)*V[i,n] + κ*V[i-1,n]
        end
        V[N,n+1] = 0
    end
    return t, x, V
end

wave_fixed (generic function with 1 method)

In [10]:
function wave_free(u0, v0, c, L, tf; N = 10, MinM = 1)
    h = L/(N-1)
    M = max(ceil(Int, c*tf/h) + 1, MinM)
    δ = tf/M
    κ = (c*δ/h)^2
    
    x = linspace(0, L, N)
    t = linspace(0, tf, M+1)
    V = zeros(N, M+1)
    
    # Initial position
    for i = 1:N
        V[i,1] = u0(x[i])
    end
    # Initial velocity
    V[1,2] = δ*v0(x[1]) + κ*V[2,1] + (1-κ)*V[1,1]
    for i = 2:N-1
        V[i,2] = δ*v0(x[i]) + (κ*V[i+1,1] + 2(1-κ)*V[i,1] + κ*V[i-1,1])/2
    end
    V[N,2] = δ*v0(x[N]) + (1-κ)*V[N,1] + κ*V[N-1,1]

    for n = 2:M
        i = 1
        V[1,n+1] = -V[i,n-1] + 2κ*V[i+1,n] + 2(1-κ)*V[i,n]
        for i = 2:N-1
            V[i,n+1] = -V[i,n-1] + κ*V[i+1,n] + 2(1-κ)*V[i,n] + κ*V[i-1,n]
        end
        i = N
        V[N,n+1] = -V[i,n-1] + 2(1-κ)*V[i,n] + 2κ*V[i-1,n]
    end
    return t, x, V
end

wave_free (generic function with 2 methods)

In [4]:
function integral(f, a, b; N = 1000)
    I = 0.0
    h = (b-a)/N
    t = a + h
    for i = 2:N-1
        I += f(t)
        t += h
    end
    return (2I + f(a) + f(b))*h/2
end

integral (generic function with 1 method)

In [16]:
c = 1.0
L = 15
u0(x) = exp(-(x-0.5)^2*50)
#u0(x) = 0
v0(x) = 0
#v0(x) = u0(x)
#v0(x) = exp(-(x-1.5)^2*50)
#u0(x) = sin(2x*π/L)
tf = 10
N = 100
fps = 15
MinM = ceil(Int, fps*tf)

t, x, V = wave_fixed(u0, v0, c, L, tf, N=N, MinM=MinM);

M = length(t)
step = ceil(Int, M/(tf*fps))
println("M = $M")
println("step = $step")

anim = Animation()

umin, umax = extrema(V)
λ = umax - umin
umin -= 0.1λ
umax += 0.1λ

sol(t,x) = 0.5*(u0(x + c*t) + u0(x - c*t)) + 0.5*integral(v0, x - c*t, x + c*t)/c

for s = 1:step:M
    plot(x, V[:,s], leg=false)
    plot!(x->sol(t[s], x), 0, L)
    ylims!(umin, umax)
    title!("$(round(t[s],2)) s")
    frame(anim)
end
gif(anim, "tmp.gif", fps=fps)

M = 151
step = 2


INFO: Saved animation to /home/abel/projetos/pessoal/abelsiqueira.github.io/disciplinas/cm116/tmp.gif


Plots.AnimatedGif("/home/abel/projetos/pessoal/abelsiqueira.github.io/disciplinas/cm116/tmp.gif")

In [14]:

c = 1
L = 15
u0(x) = exp(-(x-0.5L)^2*5)
#u0(x) = cos(2π*x/L)
#u0(x) = sum([cos(i*π*x/L) for i = 1:10])
v0(x) = u0(x)
tf = 10
N = 100
fps = 15
MinM = ceil(Int, fps*tf)

t, x, V = wave_free(u0, v0, c, L, tf, N=N, MinM=MinM);

fps = 15
M = length(t)
step = ceil(Int, M/(tf*fps))
println("step")

anim = Animation()

umin, umax = extrema(V)
λ = umax - umin
umin -= 0.1λ
umax += 0.1λ

sol(t,x) = 0.5*(u0(x + c*t) + u0(x - c*t)) + 0.5*integral(v0, x - c*t, x + c*t)/c

for s = 1:step:M
    plot(x, V[:,s])
    plot!(x->sol(t[s], x), 0, L)
    ylims!(umin, umax)
    title!("$(round(t[s],2)) s")
    frame(anim)
end
gif(anim, "tmp.gif", fps=fps)

step


INFO: Saved animation to /home/abel/projetos/pessoal/abelsiqueira.github.io/disciplinas/cm116/tmp.gif


Plots.AnimatedGif("/home/abel/projetos/pessoal/abelsiqueira.github.io/disciplinas/cm116/tmp.gif")